In [1]:
import pandas as pd

df_meta = pd.read_csv("../query_stockmarket/origin_dataset/stockinfo_query.csv")


In [2]:
df_meta

,Nasdaq Traded,Symbol,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,NextShares,Company Description
0,Y,AAAU,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,Perth Mint Physical Gold ETF offers investors ...
1,Y,AADR,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,AdvisorShares Dorsey Wright ADR ETF offers inv...
2,Y,AAME,Q,G,N,100.0,N,N,N,Atlantic American Corporation provides a range...
3,Y,AAWW,Q,Q,N,100.0,N,N,N,Atlas Air Worldwide Holdings specializes in pr...
4,Y,AAXJ,Q,G,Y,100.0,N,N,N,iShares MSCI All Country Asia ex Japan Index F...
...,...,...,...,...,...,...,...,...,...,...
2747,Y,ZMLP,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,Direxion Zacks MLP High Income Index Shares of...
2748,Y,ZNH,N,Not applicable or not NASDAQ-listed,N,100.0,N,NaN,N,China Southern Airlines Company Limited is one...
2749,Y,ZROZ,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,The PIMCO 25 Year Zero Coupon U.S. Treasury In...
2750,Y,ZSL,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,ProShares UltraShort Silver is a financial fun...


In [32]:

# Replace missing values in the 'Market Category' column
df_meta['Market Category'] = df_meta['Market Category'].replace(['', ' '], pd.NA).fillna('Not applicable or not NASDAQ-listed')

df_meta

,Nasdaq Traded,Symbol,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,NextShares,Company Description
0,Y,AAAU,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,Perth Mint Physical Gold ETF offers investors ...
1,Y,AADR,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,AdvisorShares Dorsey Wright ADR ETF offers inv...
2,Y,AAME,Q,G,N,100.0,N,N,N,Atlantic American Corporation provides a range...
3,Y,AAWW,Q,Q,N,100.0,N,N,N,Atlas Air Worldwide Holdings specializes in pr...
4,Y,AAXJ,Q,G,Y,100.0,N,N,N,iShares MSCI All Country Asia ex Japan Index F...
...,...,...,...,...,...,...,...,...,...,...
2747,Y,ZMLP,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,Direxion Zacks MLP High Income Index Shares of...
2748,Y,ZNH,N,Not applicable or not NASDAQ-listed,N,100.0,N,NaN,N,China Southern Airlines Company Limited is one...
2749,Y,ZROZ,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,The PIMCO 25 Year Zero Coupon U.S. Treasury In...
2750,Y,ZSL,P,Not applicable or not NASDAQ-listed,Y,100.0,N,NaN,N,ProShares UltraShort Silver is a financial fun...


In [10]:
exchange_counts = df_meta["Listing Exchange"].value_counts(dropna=False)
print(exchange_counts)


Listing Exchange
P    1444
Q     710
Z     336
N     234
A      28
Name: count, dtype: int64


In [7]:
exchange_counts = df_meta["Financial Status"].value_counts(dropna=False)
print(exchange_counts)


Financial Status
NaN    2042
N       685
D        24
H         1
Name: count, dtype: int64


In [8]:
# Filter rows where Financial Status is D or H
df_troubled = df_meta[df_meta["Financial Status"].isin(["D", "H"])]

# Count Listing Exchange distribution within troubled companies
exchange_distribution = df_troubled["Listing Exchange"].value_counts(dropna=False)

print("📊 Listing Exchange distribution for Financial Status = D or H:")
print(exchange_distribution)


📊 Listing Exchange distribution for Financial Status = D or H:
Listing Exchange
Q    25
Name: count, dtype: int64


In [22]:
exchange_counts = df_meta["Market Category"].value_counts(dropna=False)
print(exchange_counts)


Market Category
     2042
G     451
Q     173
S      86
Name: count, dtype: int64


In [ ]:
from openai import AzureOpenAI
import pandas as pd
import time

client = AzureOpenAI(

)
deployment_name = "gpt-4o"

# 函数：用 GPT 改写 Security Name 为 Company Description
# ========= Step 3: Define LLM inference function ==========
def infer_company_description(security_name):
    prompt = (
    f"""You're given a security name for a publicly traded company: '{security_name}'.

Rewrite it as a natural, human-like company description that:
- Clearly explains what the company does
- Includes the company name in the sentence
- Uses varied, fluent language rather than repeating the same template
- Keeps it short (1–2 sentences), easy to understand, and engaging

Examples of acceptable variation:
- "Tesla Inc. designs cutting-edge electric vehicles and is reshaping the future of transportation."
- "Amazon.com Inc. began as an online bookstore and evolved into a global leader in cloud services and e-commerce."
- "Apple Inc. is best known for its innovative tech products like the iPhone and MacBook."

Avoid robotic repetition like “<Company> is a company that...”.

Format:
Company Description: <your output here>"""
)

    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": "You are a financial assistant that rewrites security names into clear and concise company descriptions."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=100
        )
        output = response.choices[0].message.content.strip()
        description = output.replace("Company Description:", "").strip()
        return description
    except Exception as e:
        print(f"[ERROR] {security_name}: {e}")
        return None


# 应用 GPT 调用（可以加进度条 tqdm，如果量大建议加 sleep）


descriptions = []

for name in df_meta["Security Name"]:
    description = infer_company_description(name)
    descriptions.append(description)

    # 实时可视化输出
    print(f"\n Security Name: {name}")
    print(f" Company Description: {description}\n")


# 添加新列
df_meta["Company Description"] = descriptions

# 保存结果


 Security Name: Perth Mint Physical Gold ETF
 Company Description: Perth Mint Physical Gold ETF offers investors an opportunity to buy shares backed by physical gold, providing a secure and tangible way to invest in precious metals.


 Security Name: AdvisorShares Dorsey Wright ADR ETF
 Company Description: AdvisorShares Dorsey Wright ADR ETF offers investors a dynamic way to access international markets through a diverse portfolio of American Depositary Receipts, leveraging the expertise of Dorsey Wright's investment strategies.


 Security Name: Atlantic American Corporation - Common Stock
 Company Description: Atlantic American Corporation provides a range of insurance products, specializing in life, health, and property insurance to meet diverse customer needs.


 Security Name: Atlas Air Worldwide Holdings - Common Stock
 Company Description: Atlas Air Worldwide Holdings specializes in providing air cargo and passenger charter services, playing a crucial role in global logistics 

In [2]:
import duckdb
db_path = "../query_stockmarket/query_dataset/stocktrade_query.db"
con = duckdb.connect(db_path)


tables = con.execute("SHOW TABLES").fetchdf()
table_names = tables.iloc[:,0].tolist()

print("top 10 table name:")
for name in table_names[:10]:
    print(f"📝 {name}")
print(f"\n📊 total table nums: {len(table_names)}")
try:
    df_vgr = con.execute("SELECT * FROM VGR").fetchdf()
    print(f"✅ read VGR table {len(df_vgr)} row")
    df_vgr.head()  
    print(df_vgr.head().to_string())

except Exception as e:
    print(f"❌ failed: {e}")
con.close()


前 10 个表名:
📝 AAAU
📝 AADR
📝 AAME
📝 AAWW
📝 AAXJ
📝 ABEQ
📝 ABMD
📝 ACAD
📝 ACES
📝 ACIO

📊 总表数量: 2753
✅ 读取 VGR 表，共 8186 行
         Date      Open      High       Low     Close  Adj Close   Volume
0  1987-10-08  0.000000  4.352176  4.307308  4.307308   0.044764  1791500
1  1987-10-09  4.307308  4.352176  4.307308  4.307308   0.044764   285800
2  1987-10-12  4.307308  4.352176  4.307308  4.307308   0.044764   447100
3  1987-10-13  4.307308  4.352176  4.307308  4.307308   0.044764   190500
4  1987-10-14  4.307308  4.352176  4.307308  4.352176   0.045231   287800


In [34]:

df_meta.to_csv("stockinfo_query.csv", index=False)
